参考記事

https://qiita.com/mzmttks/items/9cca88cdd430e0470c8e

In [94]:
import pulp
from collections import namedtuple, defaultdict

In [95]:
prob = pulp.LpProblem("PlusPuzzle", pulp.LpMinimize)
prob += 0

In [96]:
size = 5

numbers = range(1, size + 1) # [1, 6)
xs = range(1, size + 1)
ys = range(1, size + 1)

choices = pulp.LpVariable.dicts("Cell", (xs, ys, numbers), 0, 1, pulp.LpInteger)

In [97]:
# 1つのマスに入る値は1つだけ

for y in xs:
    for x in ys:
        prob += pulp.lpSum([choices[v][x][y] for v in numbers]) == 1

In [98]:
for v in numbers:
    for y in ys:
        prob += pulp.lpSum([choices[v][x][y] for x in xs]) == 1

for v in numbers:
    for x in xs:
        prob += pulp.lpSum([choices[v][x][y] for y in ys]) == 1


In [99]:
# 問題のブロックの定義

Block = namedtuple("Block", ("sum_number", "positions"))

blocks = [
    Block(4, [(1, 1), (2, 1), (1, 2)]),
    Block(24, [(3, 1), (4, 1), (5, 1), (2, 2), (3, 2), (4, 2)]),
    Block(5, [(5, 2), (5, 3), (4, 3)]),
    Block(7, [(2, 3), (3, 3)]),
    Block(12, [(1, 3), (1, 4), (1, 5)]),
    Block(5, [(2, 4), (2, 5)]),
    Block(4, [(3, 4), (3, 5), (4, 5)]),
    Block(14, [(4, 4), (5, 4), (5, 5)]),
]

In [100]:
def validate_blocks(blocks):
    d = defaultdict(lambda: 0)
    for b in blocks:
        for p in b.positions:
            d[p] += 1
    
    for p in zip(xs, ys):
        if d[p] != 1:
            break
    else:
        return True
    return False

validate_blocks([])

False

In [101]:
validate_blocks(blocks)

True

In [102]:
# 問題のブロックの制約を追加

for block in blocks:
    prob += pulp.lpSum([v * choices[v][x][y] for v in numbers for x, y in block.positions]) == block.sum_number

In [105]:
%%time

s = prob.solve()

CPU times: user 6.76 ms, sys: 5.02 ms, total: 11.8 ms
Wall time: 23.8 ms


In [106]:
print(pulp.LpStatus[s])

Optimal


In [107]:
for y in ys:
    for x in xs:
        for v in numbers:
            if choices[v][x][y].value() == 1:
                print(v, end=" ")
    print()

2 1 5 4 3 
1 5 4 3 2 
5 4 3 2 1 
3 2 1 5 4 
4 3 2 1 5 
